# Критерий Стьюдента. 
Имеются данные о стоимости и размерах 53940 бриллиантов:

Отделите 25% случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На обучающей выборке настройте две регрессионные модели:

линейную регрессию с помощью LinearRegression без параметров
случайный лес с помощью RandomForestRegressor с random_state=1.
Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости α=0.05?

In [2]:
from sklearn.cross_validation import train_test_split
import pandas as pd

In [6]:
data = pd.read_csv(r'C:\MyStorage\Self-Education\Coursera\Specialization\Data Structures and Algorithms\2.Data Structures\Week 2\diamonds.txt',sep = '\t')

In [8]:
price = data['price']

In [11]:
measures = data.drop('price', axis=1)

In [13]:
measure_train, measure_test, price_train,price_test = train_test_split(measures, price, test_size=0.25, random_state=1)

In [14]:
from sklearn.linear_model import LinearRegression
regr_model = LinearRegression()
regr_model.fit(measure_train, price_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
from sklearn.ensemble import RandomForestRegressor
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(measure_train, price_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=1,
           verbose=0, warm_start=False)

In [41]:
import numpy as np
regr_distr = map(np.abs,price_test-regr_model.predict(measure_test))
print np.mean(map(np.abs,price_test-regr_model.predict(measure_test)))

890.376400429


In [42]:
forest_distr = map(np.abs,price_test-forest_model.predict(measure_test))
print np.mean(map(np.abs,price_test-forest_model.predict(measure_test)))

804.702882475


Так как выборки зависимы, используем двухвыборочный критерий Стьюдента для связанных выборок

In [44]:
import scipy.stats as stats
stats.ttest_rel(regr_distr, forest_distr)

Ttest_relResult(statistic=12.745056775182125, pvalue=5.428654818173765e-37)

In [60]:
from statsmodels.stats.weightstats import *
DescrStatsW(pd.DataFrame(regr_distr)[0]-pd.DataFrame(forest_distr)[0]).tconfint_mean()

(72.497265650113349, 98.849770256267362)

## Гипотезы о долях


В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

Решение. 
1. посчитать статистику. Для несвязанных выборок


In [62]:
P = (10.+4.)/50
Z_stat = (10./34-4./16)/np.sqrt(P*(1-P)*(50./(16*34)))

 2 . посчитать достаточный уровень значимости. Нулевое распределение  - нормальное

In [65]:
import scipy.stats
1-scipy.stats.norm.cdf(Z_stat)

0.37293045872523534